<center>

# **Đồ án cuối kì**

# **Môn Lập trình cho khoa học dữ liệu**

## **Chủ đề: Các vụ án mạng ở Mỹ từ năm 2016 đến năm 2020**

</center>

<center>

# **Mục lục**

</center>

# **Các thư viện sử dụng**

In [3]:
# Libraries and Modules used
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


---

# **Mô tả tập dữ liệu**

- Tập dữ liệu này được thu thập từ [**Murder Accountability Project**](https://www.murderdata.org/p/data-docs.html), do Thomas Hargrove sáng lập và công bố. Nhìn chung, tập dữ liệu mô tả về các vụ án giết người được xảy ra ở khắp mọi bang trên nước Mỹ từ năm 1976 đến năm 2020.

- Tập dữ liệu này là một tập dữ liệu mở được công bố dưới bản quyền [***Creative Commons CC-BY license***](https://creativecommons.org/licenses/by/4.0/), cho phép sử dụng không hạn chế dưới mọi hình thức, miễn là người dùng giữ nguyên tập dữ liệu gốc.

- Tập dữ liệu bao gồm những vụ án mạng được tổng hợp từ báo cáo của FBI và dữ liệu của Freedom of Information Act về các vụ án mạng không được báo cáo cho Justice Department.

- Do tập dữ liệu có kích cỡ quá lớn (khoảng 290MB) nên nhóm chúng em chỉ lọc lấy những vụ án từ năm 2016 đến năm 2020.

---

# **Tiền xử lý dữ liệu**

## Đọc dữ liệu vào Dataframe

In [ ]:
homi_df = pd.read_csv('homicide.csv')
homi_df

## Số dòng và số cột của tập dữ liệu

In [ ]:
duplicated = homi_df.duplicated()
duplicated_line = homi_df[duplicated == True]
print('Số dòng bị trùng lắp:', len(duplicated_line))
duplicated_line

## Ý nghĩa của mỗi dòng

## Kiểm tra các dòng bị trùng lắp

## Ý nghĩa của mỗi cột

## Kiểm tra kiểu dữ liệu của các cột và điều chỉnh cho phù hợp

## Tìm sự phân phối của các cột dữ liệu kiểu số

## Tìm sự phân phối của các cột dữ liệu kiểu phân loại

## Xử lý các dòng bị thiếu dữ liệu

---

# **Phân tích dữ liệu**

## **Câu hỏi số 1**

- (Nhập câu hỏi)
- (Lợi ích của việc tìm câu trả lời cho câu hỏi này)

### Tiền xử lý

### Phân tích

### Trực quan hóa

### Trả lời câu hỏi

- (Nhập câu trả lời và giải thích)

---

## **Câu hỏi số 2**

- (Nhập câu hỏi)
- (Lợi ích của việc tìm câu trả lời cho câu hỏi này)

### Tiền xử lý

### Phân tích

### Trực quan hóa

### Trả lời câu hỏi

- (Nhập câu trả lời và giải thích)

---

## **Câu hỏi số 3**

- (Nhập câu hỏi)
- (Lợi ích của việc tìm câu trả lời cho câu hỏi này)

### Tiền xử lý

### Phân tích

### Trực quan hóa

### Trả lời câu hỏi

- (Nhập câu trả lời và giải thích)

---

## **Câu hỏi số 4**

- (Nhập câu hỏi)
- (Lợi ích của việc tìm câu trả lời cho câu hỏi này)

### Tiền xử lý

### Phân tích

### Trực quan hóa

### Trả lời câu hỏi

- (Nhập câu trả lời và giải thích)

---

# **Đôi điều về đồ án**

- Mỗi thành viên gặp khó khăn gì?
- Mỗi thành viên học được những gì?
- Nếu có thêm thời gian, sẽ làm những gì?

---

# **Các nguồn tham khảo**

https://www.kaggle.com/code/nevil7/homicide-data-identifying-the-serial-killers